<h1>Logistic Regression with distributed representation word embedding</h1>

In [110]:
import numpy as np
import pandas as pd
from scipy.sparse import find
import gensim
import tensorflow as tf

In [111]:
train = pd.read_csv('ratings_train.txt', delimiter='\t')
test = pd.read_csv('ratings_test.txt', delimiter='\t')
y_train = train.label.values
y_test = test.label.values

In [112]:
model = gensim.models.Word2Vec.load('./vectors.bin')

In [113]:
word_vectors = model.wv

In [114]:
train_segs = np.load('train_segs.npy')
test_segs = np.load('test_segs.npy')

In [115]:
word = []
for sent in train_segs:
    for seg in sent:
        word.append(seg)

In [152]:
word2num = {w:(i+1) for i,w in enumerate(set(word))}
num2word = {(i+1):w for i,w in enumerate(set(word))}

In [153]:
embedding_vector = [np.zeros(shape=300)]
for i, w in enumerate(word2num.keys()):
    if w not in word_vectors.vocab:
        embedding_vector += [np.random.normal(scale=1e-2, size=300)]
    else:
        embedding_vector += [word_vectors[w]]

In [154]:
len(embedding_vector)

61590

In [151]:
len(word_vectors.vocab)

34371

In [139]:
word2num['<UNK>'] = 0
num2word[0] = '<UNK>'

In [145]:
def sent2index(segs):
    idx = []
    for sent in segs:
        temp = []
        for seg in sent:
            if seg not in word2num.keys():
                seg = '<UNK>'
            temp.append(word2num[seg])
        idx.append(temp)
    return idx

In [146]:
train_idx = sent2index(train_segs)
test_idx = sent2index(test_segs)

In [147]:
def idx2EmbedSum(idxSet):
    sum_w2v = []
    for idxes in idxSet:
        temp = np.zeros(shape=300)
        for idx in idxes:
            temp += embedding_vector[idx]
        sum_w2v.append(temp)
    return sum_w2v

In [148]:
X_train_w2v = idx2EmbedSum(train_idx)
X_test_w2v = idx2EmbedSum(test_idx)

IndexError: list index out of range

In [123]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(X_train_w2v, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [124]:
print(logReg.score(X_train_w2v, y_train))
print(logReg.score(X_test_w2v, y_test))

0.6816333333333333
0.67722


In [125]:
n_output = 2
n_embedding = 300
n_hidden = 128
learning_rate = 1e-3
n_epoch = 10
batch_size = 64

In [126]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, n_embedding])
Y = tf.placeholder(tf.int32, shape=[None])

In [127]:
W1 = tf.Variable(tf.random_normal([n_embedding, n_hidden]))
b1 = tf.Variable(tf.random_normal([n_hidden]))
W2 = tf.Variable(tf.random_normal([n_hidden, n_output]))
b2 = tf.Variable(tf.random_normal([n_output]))

In [128]:
h = tf.nn.relu(tf.matmul(X, W1) + b1)
logits = tf.matmul(h, W2) + b2
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
prediction = tf.cast(tf.argmax(hypothesis, 1), tf.int32)
correct_prediction = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [129]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(n_epoch):
    total_batch = int(len(X_train_w2v) / batch_size)
    cost_avg = 0
    print('< epoch :', (epoch+1), '>')
    for i in range(total_batch):
        if i == (total_batch-1):
            batch_xs = X_train_w2v[(i*batch_size):len(X_train_w2v)]
            batch_ys = y_train[(i*batch_size):len(y_train)]
        else:
            batch_xs = X_train_w2v[i*batch_size:(i+1)*batch_size]
            batch_ys = y_train[i*batch_size:(i+1)*batch_size]       
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        cost_avg += cost_val
        if i % 1000 == 999:
            print('%04d' % (i+1), 'Cost: ', '{:.3f}'.format(cost_avg/500))
            cost_avg = 0

< epoch : 1 >
1000 Cost:  11.374
2000 Cost:  5.997
< epoch : 2 >
1000 Cost:  4.369
2000 Cost:  3.630
< epoch : 3 >
1000 Cost:  3.042
2000 Cost:  2.738
< epoch : 4 >
1000 Cost:  2.404
2000 Cost:  2.274
< epoch : 5 >
1000 Cost:  2.074
2000 Cost:  2.002
< epoch : 6 >
1000 Cost:  1.865
2000 Cost:  1.826
< epoch : 7 >
1000 Cost:  1.733
2000 Cost:  1.709
< epoch : 8 >
1000 Cost:  1.636
2000 Cost:  1.622
< epoch : 9 >
1000 Cost:  1.563
2000 Cost:  1.555
< epoch : 10 >
1000 Cost:  1.509
2000 Cost:  1.502
< epoch : 11 >
1000 Cost:  1.461
2000 Cost:  1.460
< epoch : 12 >
1000 Cost:  1.425
2000 Cost:  1.424
< epoch : 13 >
1000 Cost:  1.393
2000 Cost:  1.395
< epoch : 14 >
1000 Cost:  1.372
2000 Cost:  1.372
< epoch : 15 >
1000 Cost:  1.352
2000 Cost:  1.348
< epoch : 16 >
1000 Cost:  1.333
2000 Cost:  1.331
< epoch : 17 >
1000 Cost:  1.317
2000 Cost:  1.317
< epoch : 18 >
1000 Cost:  1.302
2000 Cost:  1.301
< epoch : 19 >
1000 Cost:  1.292
2000 Cost:  1.291
< epoch : 20 >
1000 Cost:  1.279
2000 C

In [130]:
test_batch = int(len(X_test_w2v) / batch_size)
test_acc = 0
for i in range(test_batch):
    if i == (test_batch-1):
        batch_xs = X_test_w2v[(i*batch_size):len(X_test_w2v)]
        batch_ys = y_test[(i*batch_size):y_test.shape[0]]
    else:
        batch_xs = X_test_w2v[i*batch_size:(i+1)*batch_size]
        batch_ys = y_test[i*batch_size:(i+1)*batch_size]       
    acc = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys})
    test_acc += acc
print('Accuracy: ', '{:.3f}'.format(test_acc/test_batch))

Accuracy:  0.699
